In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys
import os
sys.path.append(os.path.abspath('./odibi_de_v2'))
os.environ["PYTHONDONTWRITEBYTECODE"] = "1"
from odibi_de_v2.transformer import PandasSteamPropertyExtractor
from odibi_de_v2.config import ConfigUtils
from odibi_de_v2.utils import send_email_using_logic_app
from odibi_de_v2.databricks import run_notebook_with_logging,log_to_centralized_table

In [0]:
import pandas as pd
# from odibi_de_v2.transformer.pandas_steam_property_extractor import PandasSteamPropertyExtractor

# Sample data with temperature in Celsius
df = pd.DataFrame({
    "Timestamp": pd.date_range("2024-01-01", periods=5, freq="H"),
    "Boiler_Pressure": [5.0, 6.2, 4.8, 7.0, 5.5],       # MPa
    "Feedwater_Temp": [100.0, 95.0, 105.0, 98.0, 102.0]  # °C
})

# Define transformer with T in Kelvin
transformer = PandasSteamPropertyExtractor(
    input_params={
        "P": "Boiler_Pressure",                      # Use pressure column as-is
        "T": 'lambda row: row["Feedwater_Temp"] + 273.15'  # Convert °C to K
    },
    output_properties=["h", "s", "cp", "cv"],
    prefix="Feedwater")

# Transform
df_transformed = transformer.transform(df)

# Display output
display(df_transformed)
